# Chapter 1: MCMC基礎理論

## 学習目標
- MCMCの基本概念と必要性を理解する
- マルコフ連鎖の基本性質を学ぶ
- 定常分布と詳細釣り合い条件を理解する
- 確率分布からのサンプリング問題を把握する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.integrate import quad
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

## 1.0 なぜMCMCなのか？ - ベイズ推論の「積分地獄」からの脱出

現代のデータサイエンスにおいて、**不確実性の定量化**は極めて重要です。従来の点推定だけでは「この予測にどの程度の信頼性があるのか？」という疑問に答えることができません。

### ベイズ推論：理想と現実のギャップ

ベイズ推論は、不確実性を確率分布として表現する理論的に美しい枠組みです：

$$p(\theta|X) = \frac{p(X|\theta)p(\theta)}{p(X)}$$

しかし、この美しい式には**計算上の悪夢**が隠されています。

### 積分地獄：現実の壁

分母の **エビデンス** $p(X)$ を計算するには、以下の積分が必要です：

$$p(X) = \int p(X|\theta)p(\theta)d\theta$$

**なぜこれが「地獄」なのか？**

1. **高次元の呪い**: パラメータが10個あれば10次元積分
2. **複雑な関数**: 指数関数や対数の組み合わせ
3. **解析解の不存在**: ほとんどの実用的なモデルで解析解がない
4. **数値積分の限界**: 高次元では格子点法が破綻

### 具体例：シンプルなモデルでも困難

たった3パラメータの線形回帰でも：
$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \epsilon$$

事後分布を求めるには **3次元積分** が必要になり、数値積分は現実的ではありません。

## 1.1 MCMCによる革命：「計算せずしてサンプリングする」

MCMCは、この積分問題を**正面から解くことを潔く放棄**します。

### 革命的発想の転換

❌ **従来**: 「分布の式を正確に求めて、そこから統計量を計算」
✅ **MCMC**: 「分布に従うサンプルを大量生成して、サンプルから統計量を近似」

### なぜサンプリングが有効なのか？

**大数の法則**により、サンプルが十分多ければ：
- サンプル平均 → 母平均
- サンプル分散 → 母分散  
- サンプル分位点 → 母分位点

**重要な洞察**: 分布の正確な式が分からなくても、その分布に従うサンプルがあれば、あらゆる統計的推論が可能！

## 1.2 ベイズ推論の壁とMCMCの革命

**MCMC（Markov Chain Monte Carlo）**は、複雑な確率分布からサンプリングを行うための強力な手法です。直接サンプリングが困難な高次元の確率分布から、マルコフ連鎖を用いて間接的にサンプルを生成します。

### MCMCが必要な理由
- 正規化定数の計算が困難な分布からのサンプリング
- 高次元空間での積分計算
- ベイズ推論における事後分布からのサンプリング

### MCMCの真の役割

MCMCは単なる計算手法ではありません。これは：

1. **ベイズ統計学を実用化した革命的技術**
2. **理論と実践を繋ぐ橋渡し**
3. **現代データサイエンスの基盤技術**

現在、物理学、生物学、経済学、機械学習など、MCMCが活用されていない科学分野を見つけることは困難です。

### 本章の学習目標

この革命的な手法の背景にある理論を理解し、実装できるようになることが本章の目標です：

- マルコフ連鎖の基本性質
- 定常分布と詳細釣り合い条件  
- MCMCアルゴリズムの設計原理

### 例：正規化定数が不明な分布

以下のような分布を考えてみましょう：
$$p(x) = \frac{1}{Z} \exp\left(-\frac{x^4}{4} + \frac{x^2}{2}\right)$$

ここで$Z$は正規化定数ですが、これを解析的に計算するのは困難です。

In [ ]:
def unnormalized_pdf(x):
    """正規化されていない確率密度関数"""
    return np.exp(-x**4/4 + x**2/2)

# 数値積分で正規化定数を近似計算
Z, _ = quad(unnormalized_pdf, -10, 10)
print(f"正規化定数 Z ≈ {Z:.4f}")

def normalized_pdf(x):
    return unnormalized_pdf(x) / Z

# 可視化
x = np.linspace(-3, 3, 1000)
y_unnorm = unnormalized_pdf(x)
y_norm = normalized_pdf(x)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(x, y_unnorm, 'b-', linewidth=2)
ax1.set_title('Unnormalized Distribution')
ax1.set_xlabel('x')
ax1.set_ylabel('f(x)')

ax2.plot(x, y_norm, 'r-', linewidth=2)
ax2.set_title('Normalized Distribution')
ax2.set_xlabel('x')
ax2.set_ylabel('p(x)')

plt.tight_layout()
plt.show()

## 1.2 マルコフ連鎖の基本

**マルコフ連鎖**は、MCMCの核となる概念です。

### マルコフ性
現在の状態のみに依存して次の状態が決まる性質：
$$P(X_{t+1} | X_t, X_{t-1}, ..., X_0) = P(X_{t+1} | X_t)$$

### 遷移確率行列
状態$i$から状態$j$への遷移確率を$p_{ij}$とすると：
$$P = \begin{pmatrix}
p_{11} & p_{12} & \cdots & p_{1n} \\
p_{21} & p_{22} & \cdots & p_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
p_{n1} & p_{n2} & \cdots & p_{nn}
\end{pmatrix}$$

### 例：簡単なマルコフ連鎖のシミュレーション

In [ ]:
def simulate_markov_chain(P, initial_state, n_steps):
    """
    マルコフ連鎖のシミュレーション
    
    Parameters:
    - P: 遷移確率行列
    - initial_state: 初期状態
    - n_steps: ステップ数
    """
    n_states = P.shape[0]
    states = np.zeros(n_steps, dtype=int)
    states[0] = initial_state
    
    for t in range(1, n_steps):
        current_state = states[t-1]
        # 現在の状態から次の状態への遷移確率に基づいてサンプリング
        states[t] = np.random.choice(n_states, p=P[current_state])
    
    return states

# 3状態のマルコフ連鎖の例
P = np.array([
    [0.7, 0.2, 0.1],  # 状態0からの遷移確率
    [0.3, 0.4, 0.3],  # 状態1からの遷移確率
    [0.2, 0.3, 0.5]   # 状態2からの遷移確率
])

# シミュレーション実行
chain = simulate_markov_chain(P, initial_state=0, n_steps=1000)

# 結果の可視化
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# 状態の時系列
ax1.plot(chain[:200], 'o-', markersize=3, linewidth=1)
ax1.set_title('Markov Chain State Transitions (first 200 steps)')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('State')
ax1.set_yticks([0, 1, 2])
ax1.grid(True, alpha=0.3)

# 状態の分布
state_counts = np.bincount(chain)
state_probs = state_counts / len(chain)

ax2.bar(range(3), state_probs, alpha=0.7, color=['blue', 'orange', 'green'])
ax2.set_title('Empirical State Distribution')
ax2.set_xlabel('State')
ax2.set_ylabel('Probability')
ax2.set_xticks([0, 1, 2])

plt.tight_layout()
plt.show()

print(f"経験的状態分布: {state_probs}")

## 1.3 定常分布

**定常分布**$\pi$は、以下の条件を満たす分布です：
$$\pi = \pi P$$

これは、分布$\pi$がマルコフ連鎖の長期的な振る舞いを表すことを意味します。

In [ ]:
def find_stationary_distribution(P, method='eigenvalue'):
    """
    定常分布の計算
    
    Parameters:
    - P: 遷移確率行列
    - method: 'eigenvalue' または 'power_iteration'
    """
    if method == 'eigenvalue':
        # 固有値・固有ベクトル法
        eigenvals, eigenvecs = np.linalg.eig(P.T)
        # 固有値1に対応する固有ベクトルを探す
        idx = np.argmin(np.abs(eigenvals - 1.0))
        stationary = np.real(eigenvecs[:, idx])
        stationary = stationary / np.sum(stationary)  # 正規化
        return stationary
    
    elif method == 'power_iteration':
        # べき乗法による近似
        pi = np.ones(P.shape[0]) / P.shape[0]  # 初期分布
        for _ in range(1000):
            pi_new = pi @ P
            if np.allclose(pi, pi_new, atol=1e-10):
                break
            pi = pi_new
        return pi

# 先ほどの遷移行列Pの定常分布を計算
stationary_exact = find_stationary_distribution(P, method='eigenvalue')
stationary_approx = find_stationary_distribution(P, method='power_iteration')

print("理論的定常分布 (固有値法):")
print(f"π = [{stationary_exact[0]:.4f}, {stationary_exact[1]:.4f}, {stationary_exact[2]:.4f}]")

print("\n近似定常分布 (べき乗法):")
print(f"π = [{stationary_approx[0]:.4f}, {stationary_approx[1]:.4f}, {stationary_approx[2]:.4f}]")

print(f"\n経験分布との比較:")
print(f"経験分布: [{state_probs[0]:.4f}, {state_probs[1]:.4f}, {state_probs[2]:.4f}]")
print(f"理論値:   [{stationary_exact[0]:.4f}, {stationary_exact[1]:.4f}, {stationary_exact[2]:.4f}]")

## 1.4 詳細釣り合い条件：MCMCの設計原理

**詳細釣り合い条件**（Detailed Balance Condition）は、MCMCアルゴリズムの設計において重要な概念です：

$$\pi(x) P(x \rightarrow y) = \pi(y) P(y \rightarrow x)$$

### 物理学的直感：平衡状態の流れ

この条件は、非常に直感的なイメージで理解できます。系が平衡状態（定常分布π）にあるとき：

- **左辺**: 状態xから状態yへの「確率的な流れ」
- **右辺**: 状態yから状態xへの「確率的な流れ」

これらが等しいということは、**すべての状態ペア間で流れが釣り合っている**ことを意味します。

### 人口移動のアナロジー：国家間の移住バランス

2つの国A・Bの人口移動で考えてみましょう：

**設定**:
- π(A), π(B): 各国の定常的な人口
- T(A→B), T(B→A): 各国からの移住率

**詳細釣り合い条件**は：
「A国からB国への年間移住者数」=「B国からA国への年間移住者数」

$$\underbrace{\pi(A) \times T(A \rightarrow B)}_{\text{A→Bの移住者数}} = \underbrace{\pi(B) \times T(B \rightarrow A)}_{\text{B→Aの移住者数}}$$

### 🌍 拡張例：多国間の移住システム

In [ ]:
def visualize_detailed_balance():
    """詳細釣り合い条件の直感的理解のための可視化"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # 左図：流れが釣り合っていない場合
    ax1.set_xlim(0, 10)
    ax1.set_ylim(0, 8)
    ax1.set_title('❌ 詳細釣り合いが成立しない場合', fontsize=14, color='red')
    
    # 国A, B, Cを表現
    countries = {'A': (2, 6), 'B': (8, 6), 'C': (5, 2)}
    populations = {'A': 1000, 'B': 800, 'C': 1200}
    
    for country, (x, y) in countries.items():
        circle = plt.Circle((x, y), 0.8, color='lightblue', alpha=0.7)
        ax1.add_patch(circle)
        ax1.text(x, y, f'{country}\n{populations[country]}人', 
                ha='center', va='center', fontweight='bold')
    
    # 不均衡な流れを矢印で表現
    flows_unbalanced = [
        (countries['A'], countries['B'], 50, 30),  # A→B: 50人, B→A: 30人
        (countries['B'], countries['C'], 40, 60),  # B→C: 40人, C→B: 60人
        (countries['A'], countries['C'], 20, 25),  # A→C: 20人, C→A: 25人
    ]
    
    for (start, end, flow1, flow2) in flows_unbalanced:
        # 行きの矢印
        ax1.annotate('', xy=end, xytext=start,
                    arrowprops=dict(arrowstyle='->', color='red', lw=2,
                                  connectionstyle="arc3,rad=0.1"))
        # 帰りの矢印
        ax1.annotate('', xy=start, xytext=end,
                    arrowprops=dict(arrowstyle='->', color='blue', lw=2,
                                  connectionstyle="arc3,rad=0.1"))
        
        # 流れの数値
        mid_x, mid_y = (start[0] + end[0])/2, (start[1] + end[1])/2
        ax1.text(mid_x+0.3, mid_y+0.3, f'{flow1}', color='red', fontweight='bold')
        ax1.text(mid_x-0.3, mid_y-0.3, f'{flow2}', color='blue', fontweight='bold')
    
    ax1.text(5, 0.5, '⚠️ 人口変化が発生！\n系が不安定', 
             ha='center', fontsize=12, color='red',
             bbox=dict(boxstyle="round,pad=0.3", facecolor='yellow', alpha=0.7))
    
    # 右図：詳細釣り合いが成立している場合
    ax2.set_xlim(0, 10)
    ax2.set_ylim(0, 8)
    ax2.set_title('✅ 詳細釣り合いが成立する場合', fontsize=14, color='green')
    
    for country, (x, y) in countries.items():
        circle = plt.Circle((x, y), 0.8, color='lightgreen', alpha=0.7)
        ax2.add_patch(circle)
        ax2.text(x, y, f'{country}\n{populations[country]}人', 
                ha='center', va='center', fontweight='bold')
    
    # 均衡した流れ
    flows_balanced = [
        (countries['A'], countries['B'], 40, 40),  # A⇄B: 各40人
        (countries['B'], countries['C'], 50, 50),  # B⇄C: 各50人  
        (countries['A'], countries['C'], 25, 25),  # A⇄C: 各25人
    ]
    
    for (start, end, flow1, flow2) in flows_balanced:
        # 双方向矢印
        ax2.annotate('', xy=end, xytext=start,
                    arrowprops=dict(arrowstyle='<->', color='green', lw=3,
                                  connectionstyle="arc3,rad=0"))
        
        # 均衡した流れの数値
        mid_x, mid_y = (start[0] + end[0])/2, (start[1] + end[1])/2
        ax2.text(mid_x, mid_y+0.5, f'{flow1}人', ha='center', color='green', 
                fontweight='bold', bbox=dict(boxstyle="round,pad=0.1", 
                facecolor='white', alpha=0.8))
    
    ax2.text(5, 0.5, '✅ 人口安定！\n定常状態達成', 
             ha='center', fontsize=12, color='green',
             bbox=dict(boxstyle="round,pad=0.3", facecolor='lightgreen', alpha=0.7))
    
    # 凡例
    from matplotlib.patches import Patch
    red_patch = Patch(color='red', label='一方向の流れ')
    blue_patch = Patch(color='blue', label='逆方向の流れ')
    green_patch = Patch(color='green', label='釣り合った流れ')
    
    ax1.legend(handles=[red_patch, blue_patch], loc='upper right')
    ax2.legend(handles=[green_patch], loc='upper right')
    
    for ax in [ax1, ax2]:
        ax.set_aspect('equal')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# 可視化の実行
visualize_detailed_balance()

print("🔄 局所的釣り合い → 大域的安定")
print("="*50)
print("この「局所的な」釣り合いが全ての国ペアについて成り立っていれば：")
print("1. どの国の人口も増えも減りもしない")
print("2. 系全体として人口分布が安定する（定常状態）")
print("3. 初期の人口分布によらず、最終的に同じ分布に収束")

print("\n🎯 MCMCの「神の視点」")
print("="*50)
print("❓ 通常: 「与えられた遷移法則から、運命（定常分布）を予測する」")
print("🎯 MCMC: 「望む運命（目標分布）を設定し、その運命にたどり着くような遷移法則を設計する」")
print("\nこれは、確率過程の必然性を逆手に取った、まさに「神の視点」に立ったアプローチです。")

def check_detailed_balance(P, pi):
    """
    詳細釣り合い条件のチェック
    """
    n = P.shape[0]
    detailed_balance_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            # π(i) * P(i→j) と π(j) * P(j→i) を比較
            left_side = pi[i] * P[i, j]
            right_side = pi[j] * P[j, i]
            detailed_balance_matrix[i, j] = left_side - right_side
    
    return detailed_balance_matrix

# 詳細釣り合い条件をチェック
db_matrix = check_detailed_balance(P, stationary_exact)

print("詳細釣り合い条件のチェック:")
print("π(i)P(i→j) - π(j)P(j→i) の値:")
print(db_matrix)
print(f"\n最大絶対誤差: {np.max(np.abs(db_matrix)):.6f}")

# この遷移行列は詳細釣り合い条件を満たさない（一般的な場合）
print("\n注意: 一般的なマルコフ連鎖では詳細釣り合い条件は成立しません。")
print("MCMCアルゴリズムでは、この条件を満たすように遷移確率を設計します。")

print("\n### 物理学との深い繋がり")
print("="*50)
print("詳細釣り合い条件は、統計物理学における「可逆性（reversibility）」の原理と深く関連:")
print("• 熱平衡状態では、全ての微視的プロセス（分子衝突など）がその逆プロセスと釣り合っている")
print("• MCMCアルゴリズムは、この物理的直感を数学的に実装したもの")
print("• MCMCの強力さの背景には、自然界の平衡原理との深い繋がりがある")

In [ ]:
def estimate_pi_monte_carlo(num_points):
    """
    モンテカルロ法による円周率の推定
    """
    # 正方形内にランダムな点を生成 (-1 ≤ x,y ≤ 1)
    points = np.random.uniform(-1, 1, (num_points, 2))
    
    # 各点が円の内側にあるかを判定 (x² + y² ≤ 1)
    distances_squared = np.sum(points**2, axis=1)
    inside_circle = distances_squared <= 1
    
    # π の推定値を計算
    pi_estimate = 4 * np.sum(inside_circle) / num_points
    
    return pi_estimate, points, inside_circle

# 異なるサンプル数での推定精度を比較
sample_sizes = [100, 1000, 10000, 100000]
estimates = []

print("サンプル数と円周率推定値:")
print("-" * 30)
for n in sample_sizes:
    pi_est, _, _ = estimate_pi_monte_carlo(n)
    estimates.append(pi_est)
    error = abs(pi_est - np.pi)
    print(f"N = {n:6d}: π ≈ {pi_est:.4f} (誤差: {error:.4f})")

print(f"\n真の値: π = {np.pi:.4f}")

# 可視化用に10000点でシミュレーション
pi_visual, points_visual, inside_visual = estimate_pi_monte_carlo(10000)

# 結果の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 点の分布をプロット
ax1.scatter(points_visual[inside_visual, 0], points_visual[inside_visual, 1], 
           c='blue', s=0.5, alpha=0.6, label=f'円内 ({np.sum(inside_visual)}点)')
ax1.scatter(points_visual[~inside_visual, 0], points_visual[~inside_visual, 1], 
           c='red', s=0.5, alpha=0.6, label=f'円外 ({np.sum(~inside_visual)}点)')

# 円の境界を描画
theta = np.linspace(0, 2*np.pi, 100)
circle_x = np.cos(theta)
circle_y = np.sin(theta)
ax1.plot(circle_x, circle_y, 'k-', linewidth=2)

ax1.set_xlim(-1.1, 1.1)
ax1.set_ylim(-1.1, 1.1)
ax1.set_aspect('equal')
ax1.set_title(f'モンテカルロ法による円周率推定\nπ ≈ {pi_visual:.4f}')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 推定精度の改善を表示
ax2.semilogx(sample_sizes, estimates, 'bo-', markersize=8, linewidth=2)
ax2.axhline(np.pi, color='red', linestyle='--', linewidth=2, label='真の値 π')
ax2.set_xlabel('サンプル数')
ax2.set_ylabel('π の推定値')
ax2.set_title('サンプル数と推定精度の関係')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nこの例が示すもの:")
print(f"• 単純な乱数による試行でも、大数の法則により精度が向上")
print(f"• 「確率を通じた代理評価」がモンテカルロ法の本質")
print(f"• MCMCはこの考え方を複雑な分布のサンプリングに応用")

## 1.6 MCMCのグランドデザイン

MCMCの全体像をまとめると以下のようになります：

### 目標と戦略
1. **目標**: 複雑な確率分布p(x)（例：ベイズ推論の事後分布）からサンプリング
2. **戦略**: 目標分布p(x)を定常分布として持つマルコフ連鎖を構築
3. **実行**: そのマルコフ連鎖を長時間シミュレーションし、サンプル系列を取得

### 核心的アイデア
- **モンテカルロ法**: 乱数を用いた確率的近似の力
- **マルコフ連鎖**: 定常分布への収束という必然性
- **詳細釣り合い条件**: 望む分布を定常分布として設計するレシピ

### 革命的な意義
MCMCは「困難な積分を回避する」逆転の発想により、ベイズ推論を理論上の存在から実践的なツールへと変貌させました。

### 演習問題

以下の遷移確率行列を持つマルコフ連鎖をシミュレートし、定常分布を求めなさい。

In [ ]:
# 演習用の遷移行列
P_exercise = np.array([
    [0.5, 0.3, 0.2],
    [0.2, 0.6, 0.2],
    [0.1, 0.4, 0.5]
])

# ここにコードを書いてください
# 1. マルコフ連鎖をシミュレート
# 2. 定常分布を理論的に計算
# 3. 経験分布と比較

# 解答例（コメントアウト）
# chain_ex = simulate_markov_chain(P_exercise, 0, 10000)
# stationary_ex = find_stationary_distribution(P_exercise)
# empirical_ex = np.bincount(chain_ex) / len(chain_ex)
# print(f"理論値: {stationary_ex}")
# print(f"経験値: {empirical_ex}")

### 問題2：収束の可視化
異なる初期状態から始めたマルコフ連鎖が定常分布に収束する様子を可視化しなさい。

In [ ]:
# ここにコードを書いてください
# 1. 複数の初期状態からマルコフ連鎖を開始
# 2. 各状態の確率の時間変化をプロット
# 3. 定常分布への収束を観察

pass  # 学習者が実装

## まとめ

この章では、MCMCの基礎となる以下の概念を学びました：

1. **MCMCの必要性**：複雑な分布からのサンプリング問題
2. **マルコフ連鎖**：マルコフ性と遷移確率行列
3. **定常分布**：長期的な振る舞いと計算方法
4. **詳細釣り合い条件**：MCMCアルゴリズム設計の原理

次の章では、これらの理論を基に具体的なMCMCアルゴリズムであるメトロポリス・ヘイスティングス法を学習します。

### 参考文献
- Robert, C. & Casella, G. (2004). Monte Carlo Statistical Methods
- Brooks, S. et al. (2011). Handbook of Markov Chain Monte Carlo